# Lab 3

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/giswqs/geog-312/blob/main/book/labs/lab_03.ipynb)

This notebook contains exercises based on the lectures on [**Functions and Classes**](https://geog-312.gishub.org/book/python/06_functions_classes.html) and [**Files and Exception Handling**](https://geog-312.gishub.org/book/python/07_files.html). These exercises will help reinforce the concepts of functions, classes, file handling, and exception management in geospatial contexts.

## Exercise 1: Calculating Distances with Functions

- Define a function `calculate_distance` that takes two geographic coordinates (latitude and longitude) and returns the distance between them using the Haversine formula.
- Use this function to calculate the distance between multiple pairs of coordinates.

In [1]:
def calculate_distance(coord1, coord2):
  # Calculate the distance between two points on the earth
  # The latitude and longitude are in degrees
  # The return value is in kilometers
  # The formula is based on the Haversine formula
  # https://en.wikipedia.org/wiki/Haversine_formula
  # The radius of the earth is 6371 km
  import math
  # Convert latitude and longitude from degrees to radians
  lat1, lat2, lon1, lon2 = map(math.radians, [coord1[0], coord2[0], coord1[1], coord2[1]])

  # Calculate the differences
  delta_latitude = lat2 - lat1
  delta_longitude = lon2 - lon1
  # Calculate the distance
  a = math.sin(delta_latitude / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(delta_longitude / 2) ** 2
  c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
  distance = 6371 * c
  return distance

calculate_distance((41.507483, -99.436554), (38.504048, -98.315949))

347.32834803942626

## Exercise 2: Batch Distance Calculation

- Create a function `batch_distance_calculation` that accepts a list of coordinate pairs and returns a list of distances between consecutive pairs.
- Test the function with a list of coordinates representing several cities.

In [2]:
def batch_distance_calculator(coords):
  # Calculate the distance between a list of coordinates
  # The input is a list of coordinates
  # The return value is a list of lists of distances
  distances = []
  for coord_pair in coords:
    distance = calculate_distance(coord_pair[0], coord_pair[1])
    distances.append(distance)

  return distances

batch_distance_calculator([[(41.507483, -99.436554), (38.504048, -98.315949)], [(41.507483, -99.436554), (38.504048, -98.315949)]])

[347.32834803942626, 347.32834803942626]

## Exercise 3: Creating and Using a Point Class

- Define a `Point` class to represent a geographic point with attributes `latitude`, `longitude`, and `name`.
- Add a method `distance_to` that calculates the distance from one point to another.
- Instantiate several `Point` objects and calculate the distance between them.

In [3]:
class Point:
  def __init__(self, lat, lon, name):
    self.lat = lat
    self.lon = lon
    self.name = name

  def __str__(self):
    return f"{self.name or 'Point'} ({self.latitude}, {self.longitude})"

  def distance_to(self, other):
    return calculate_distance((self.lat, self.lon), (other.lat, other.lon))
  
p1 = Point(41.507483, -99.436554, "Lincoln, NE")
p2 = Point(38.504048, -98.315949, "Hays, KS")

p1.distance_to(p2)

347.32834803942626

## Exercise 4: Reading and Writing Files

- Write a function `read_coordinates` that reads a file containing a list of coordinates (latitude, longitude) and returns them as a list of tuples.
- Write another function `write_coordinates` that takes a list of coordinates and writes them to a new file.
- Ensure that both functions handle exceptions, such as missing files or improperly formatted data.

In [4]:
def read_coordinates(file_name):
  # Read a file with coordinates
  # The file should have one coordinate per line
  # The coordinates should be in the format "latitude, longitude"
  # The return value is a list of coordinates
  coords = []
  try:
    with open(file_name, "r") as file:
      for line in file.readlines():
        lat, lon = line.strip().split(",")
        coords.append((float(lat), float(lon)))
  except FileNotFoundError:
    print(f"File {file_name} not found")
  except Exception as e:
    print(f"An unexpected error occurred while processing the file: {e}")
  finally:
    print(f"Finished processing {file_name}")

  return coords

def write_coordinates(file_name, coords):
  # Write a list of coordinates to a file
  # The coordinates should be in the format "latitude, longitude"
  try:
    with open(file_name, "w") as file:
      for coord in coords:
        file.write(f"{coord[0]}, {coord[1]}\n")
  except FileNotFoundError:
    print(f"File {file_name} not found")
  except Exception as e:
    print(f"An unexpected error occurred while processing the file: {e}")
  finally:
    print(f"Finished processing {file_name}")

## Exercise 5: Processing Coordinates from a File

- Create a function that reads coordinates from a file and uses the `Point` class to create `Point` objects.
- Calculate the distance between each consecutive pair of points and write the results to a new file.
- Ensure the function handles file-related exceptions and gracefully handles improperly formatted lines.

In [11]:
# Create a sample coordinates.txt file
sample_data = """41.507483, -99.436554
38.504048, -98.315949
35.6895,139.6917
34.0522,-118.2437
51.5074,-0.1278
-33.8688,151.2093
48.8566,2.3522"""

output_file = "coordinates.txt"

try:
    with open(output_file, "w") as file:
        file.write(sample_data)
    print(f"Sample file '{output_file}' has been created successfully.")
except Exception as e:
    print(f"An error occurred while creating the file: {e}")

Sample file 'coordinates.txt' has been created successfully.


In [12]:
def process_coordinates(input_file, output_file):
  try:
    with open(output_file, "w") as file:
      # Read coordinates from an input file
      coords = read_coordinates(input_file)
      # Create Points from the coordinates
      prev = None # Store temp point
      distances = [] # Store distances
      for coord in coords:
        p = Point(coord[0], coord[1], "")
        if prev is not None:
          distance = prev.distance_to(p)
          print(f"The distance between {prev.name} and {p.name} is {distance:.4f} km")
          file.write(f"{distance}\n")
        prev = p
  except FileNotFoundError:
    print(f"File {output_file} not found")
  except Exception as e:
    print(f"An unexpected error occurred while processing the file: {e}")
  finally:
    print(f"Finished creating {output_file}")

process_coordinates("coordinates.txt", "distances.txt")

Finished processing coordinates.txt
The distance between  and  is 347.3283 km
The distance between  and  is 9838.9310 km
The distance between  and  is 8815.4734 km
The distance between  and  is 8755.6023 km
The distance between  and  is 16993.9335 km
The distance between  and  is 16960.4974 km
Finished creating distances.txt


## Exercise 6: Exception Handling in Data Processing

- Modify the `batch_distance_calculation` function to handle exceptions that might occur during the calculation, such as invalid coordinates.
- Ensure the function skips invalid data and continues processing the remaining data.

In [14]:
def batch_distance_calculator(coords):
  def valid_coordinates(lat, lon):
    if lat > 90 or lat < -90 or lon > 180 or lon < -180:
      return False
    return True
  # Calculate the distance between a list of coordinates
  # The input is a list of coordinates
  # The return value is a list of lists of distances
  distances = []
  for coord_pair in coords:
    lat1, lon1 = coord_pair[0][0], coord_pair[0][1]
    lat2, lon2 = coord_pair[0][0], coord_pair[0][1]
    if not valid_coordinates(lat1, lon1) or not valid_coordinates(lat2, lon2):
      print(f"Invalid coordinates: {coord_pair}")
      continue
    distance = calculate_distance(coord_pair[0], coord_pair[1])
    distances.append(distance)

  return distances

batch_distance_calculator([[(41.507483, -99.436554), (38.504048, -98.315949)], [(41.507483, -99.436554), (38.504048, -98.315949)], [(141.507483, -99.436554), (38.504048, -98.315949)]])

Invalid coordinates: [(141.507483, -99.436554), (38.504048, -98.315949)]


[347.32834803942626, 347.32834803942626]